Übung: Rolling Update 
-----------------

In dieser Übung erstellen wir mehrere Pods ab dem gleichen Image mit jeweils einem ReplicaSet, Deployment, Service und **Ingress**.

Das passiert in einer eigenen Namespace um die Resultate gezielt Darstellen zu können:

In [ ]:
! kubectl create namespace depl

Wir Erzeugen den Pod, ReplicaSet, Deployment und Service.

Für das `Replica-Set` genügt eine Angabe `replica: 5`.

Die Pods folgen ab `spec.containers`.

Mittels `---` können mehrere Ressourcen in einer Datei zusammengefasst werden. Hier Deployment und Service.

In [ ]:
%%bash
cat <<%EOF% | kubectl --namespace depl apply -f -
apiVersion: v1
kind: Service
metadata:
  name: webshop
  labels:
    app: webshop
    group: web
    tier: frontend
spec:
  ports:
  - port: 80
    targetPort: 80  
    protocol: TCP
  selector:
    app: webshop
  type: LoadBalancer          
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: webshop
spec:
  replicas: 5
  selector:
    matchLabels:
      app: webshop
  template:
    metadata:
      labels:
        app: webshop
        group: web
        tier: frontend
    spec:
      containers:
      - name: webshop
        image: registry.gitlab.com/ch-mc-b/autoshop/shop:1.0.0
        imagePullPolicy: IfNotPresent        
        ports:
        - containerPort: 80
          name: webshop
%EOF%

Zur Kontrolle geben wir die Erzeugten Ressourcen aus:

In [ ]:
# ! kubectl get pod,deployment,replicaset,service --namespace depl -o wide
! kube-lineage --namespace depl deployment webshop -d 2 --exclude-types events 

Schauen wir uns zuerst die Ausgabe des laufenden Pods an, diesmal über den URL des Kubernetes API Servers und den via Ingress Ressourcen angehängten Prefix.

Der Prefix `frontend` ist fix in der Ingress Ressource hinterlegt, `index.html` ergibt sich wie der Container aufgebaut ist (Apache Server mit einer HTML Datei).

In [ ]:
! echo "BPMN Frontend: http://$(cat ~/work/server-ip):$(kubectl get service --namespace depl webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/frontend/index.html"

Wir wollen jedoch nicht die letzte (`1.0.0`) Version von `webshop` sondern die Version `2.0.0`, deshalb führen wir einen Rolling Update durch, bzw. Ändern die Versionsnummer hinter dem Imagenamen.

In [ ]:
! kubectl set image deployment/webshop webshop=registry.gitlab.com/ch-mc-b/autoshop/shop:1.0.0 --namespace depl

Die Änderungen können wir uns Anzeigen lassen:

In [ ]:
#! kubectl get pod,deployment,replicaset,service --namespace depl -o wide   # Unuebersichtlich!
#! kubectl describe deployment/webshop --namespace depl
! kube-lineage --namespace depl deployment webshop -d 2  

In [ ]:
! echo "BPMN Frontend: http://$(cat ~/work/server-ip):$(kubectl get service --namespace depl webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/frontend/index.html"

Neu sollte im Titel die Versionnummer `2.0.0` angezeigt werden.

- - -

RollOut
-----------

Wenn die neue Version der Software nicht wie erwartet funktoniert, können wir zur vorherigen Version zurückkehren.

Dies ist möglich, weil Kubernetes den Rollout-Verlauf von Deployment in Form von Revisionen speichert .


In [ ]:
! kubectl rollout history deployment/webshop --namespace depl

Es sollten zwei Revisionen angezeigt werden. Die aktuelle ist die Revision 2.

Um auf die vorherige Revision zurück zu kehren, verwenden wir:

In [ ]:
! kubectl rollout undo deployment/webshop --namespace depl

In [ ]:
! kubectl rollout history deployment/webshop --namespace depl

Aufräumen

In [ ]:
! kubectl delete namespace depl

***
Fragen
======

Beantwortet die Fragen einzeln oder in der Gruppe und vergleicht diese mit den Antworten

---

Was ist die Aufgabe eines ReplicaSets?
<details><summary>Antwort</summary><p>  
Stellt sicher, dass N Pods laufen sind es zu wenig, werden neue gestartet, sind es zu viele werden Pods beendet, gruppiert durch den Label Selector
</p></details>

---

Für was können Deployments verwendet werden?
<details><summary>Antwort</summary><p>  
Ermöglichen Deklarative Updates von Container Images in Pods. 
</p></details>

---